In [2]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

[[20]]
[1] TRUE

In [3]:
Asp_genus_count <- readRDS(paste0(INT_SPL, "Asp_genus_count.Rdata"))[[fav_threshold]]
Asp_species_count <- readRDS(paste0(INT_SPL, "Asp_species_count.Rdata"))
total_read_count <- readRDS(paste0(INT_SPL, "total_read_count.Rdata"))[[fav_threshold]]

In [4]:
Bctrls <- colnames(Asp_genus_count)[which(grepl(colnames(Asp_genus_count), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count), pattern = "Bctrl$"))]

Pctrls <- colnames(Asp_genus_count)[which(grepl(colnames(Asp_genus_count), pattern = "^H") & 
                                         grepl(colnames(Asp_genus_count), pattern = "Pctrl$"))]

ctrl_list <- list(Bctrls, Pctrls)
names(ctrl_list) <- c("Bctrls", "Pctrls")

In [5]:
lst_fisher_genus <- list()
for (fav_db in dbs_sel_min){
    if (fav_db == "EPRSc2"){print(fav_db); tmp_genus_threshold = "conf0.8"}
    if (fav_db == "EPRSFv64MCAspDM"){print(fav_db); tmp_genus_threshold = "conf0.9"}
        
    Asp_genus_count <- readRDS(paste0(INT_SPL, "Asp_genus_count.Rdata"))[[tmp_genus_threshold]]
    Asp_species_count <- readRDS(paste0(INT_SPL, "Asp_species_count.Rdata"))
    total_read_count <- readRDS(paste0(INT_SPL, "total_read_count.Rdata"))[[tmp_genus_threshold]]
    
    Asp_genus_count_df <- Asp_genus_count %>% 
        melt(varnames = c("db", "sample")) %>% 
        filter(db == fav_db) %>% 
        filter(grepl(sample, pattern = "asp$") | 
               grepl(sample, pattern = "ctrl$") | 
               grepl(sample, pattern = "ctrl2$") | 
               grepl(sample, pattern = "ctrl3$") | 
               grepl(sample, pattern = "VAL$")
              ) %>% 
        rename(value = "Asp_genus_count")

    total_read_count_df <- total_read_count %>% 
        melt(varnames = c("db", "sample")) %>% 
        filter(db == fav_db) %>% 
        filter(grepl(sample, pattern = "asp$") | 
               grepl(sample, pattern = "ctrl$") | 
               grepl(sample, pattern = "ctrl2$") | 
               grepl(sample, pattern = "ctrl3$") | 
               grepl(sample, pattern = "VAL$")
              ) %>% 
        rename(value = "total_read_count")

    df_fisher <- full_join(x = Asp_genus_count_df, y = total_read_count_df, by = c("db", "sample")) %>% 
        mutate(control_type = ifelse(grepl(sample, pattern = "Bctrl") | grepl(sample, pattern = "Basp"), 
                                     yes = "Bctrls", no = 
                              ifelse(grepl(sample, pattern = "Pctrl") | grepl(sample, pattern = "Pasp"), 
                                     yes = "Pctrls", no = NA))) %>% 
        mutate(origin = ifelse(control_type == "Bctrls", yes = "BAL ss-cfDNA", no = 
                        ifelse(control_type == "Pctrls", yes = "Plasma ss-cfDNA", no = NA))) %>% 
        mutate(fisher.t.ext = NA) %>%  mutate(fisher.t.int = NA) 

    ####TEMP!!!!!
    df_fisher <- df_fisher %>% 
        filter(!is.na(total_read_count) & !is.na(Asp_genus_count)) 

    ##genus level
    df_fisher_genus <- df_fisher
    for (i in 1:nrow(df_fisher_genus)){
        db_tmp <- df_fisher_genus$db[i]
        ctrl <- df_fisher_genus$control_type[i]

        tmp_ext <- c()
        for (ext_c in ctrl_list[[ctrl]]){
            if(ext_c == c(df_fisher_genus$sample[i])){next}
            ft_ext <- as.numeric(fisher.test(
                            rbind(c(df_fisher_genus$Asp_genus_count[i],
                                   (df_fisher_genus$total_read_count[i]-df_fisher_genus$Asp_genus_count[i])),
                                  c(Asp_genus_count[db_tmp, ext_c], 
                                    (total_read_count[db_tmp, ext_c])-Asp_genus_count[db_tmp, ext_c])), 
                            alternative="greater")$p.value)
            tmp_ext <- c(tmp_ext, ft_ext)
            rm(ft_ext)
        }


        if(ctrl == "Pctrls" & df_fisher_genus$sample[i] %nin% c('A05Pctrl2','A05Pctrl3')){
            int_c <- gsub(df_fisher_genus$sample[i], pattern = "asp", replacement = "ctrl")
            if(int_c %in% colnames(Asp_genus_count) & int_c %in% colnames(total_read_count)){
                df_fisher_genus$fisher.t.int[i] <- as.numeric(fisher.test(
                                rbind(c(df_fisher_genus$Asp_genus_count[i],
                                       (df_fisher_genus$total_read_count[i]-df_fisher_genus$Asp_genus_count[i])),
                                      c(Asp_genus_count[db_tmp, int_c], 
                                        (total_read_count[db_tmp, int_c])-Asp_genus_count[db_tmp, int_c])), 
                                alternative="greater")$p.value)
            }
        }
        df_fisher_genus$fisher.t.ext[i] <- mean(tmp_ext, na.rm = TRUE)
    }
    lst_fisher_genus[[fav_db]] <- df_fisher_genus
}
saveRDS(lst_fisher_genus, paste0(INT_SPL, "lst_fisher_genus.Rdata"))

[1] "EPRSc2"
[1] "EPRSFv64MCAspDM"


In [6]:
lst_fisher_species <- list()
for (fav_db in dbs_sel_min){
    if (fav_db == "EPRSc2"){print(fav_db); tmp_species_threshold = "conf0.4"}
    if (fav_db == "EPRSFv64MCAspDM"){print(fav_db); tmp_species_threshold = "conf0.2"}
    
    Asp_genus_count <- readRDS(paste0(INT_SPL, "Asp_genus_count.Rdata"))[[tmp_species_threshold]]
    Asp_species_count <- readRDS(paste0(INT_SPL, "Asp_species_count.Rdata"))
    total_read_count <- readRDS(paste0(INT_SPL, "total_read_count.Rdata"))[[tmp_species_threshold]]
    df_fisher_species <- Asp_species_count %>% 
        filter(db == fav_db) %>% 
        filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
        filter(grepl(sample, pattern = "asp$") | 
               grepl(sample, pattern = "ctrl$") | 
               grepl(sample, pattern = "ctrl2$") | 
               grepl(sample, pattern = "ctrl3$") | 
               grepl(sample, pattern = "VAL$")
              ) %>% 
        mutate(control_type = ifelse(grepl(sample, pattern = "Bctrl") | grepl(sample, pattern = "Basp"), 
                                     yes = "Bctrls", no = 
                              ifelse(grepl(sample, pattern = "Pctrl") | grepl(sample, pattern = "Pasp"), 
                                     yes = "Pctrls", no = NA))) %>% 
        mutate(origin = ifelse(control_type == "Bctrls", yes = "BAL ss-cfDNA", no = 
                        ifelse(control_type == "Pctrls", yes = "Plasma ss-cfDNA", no = NA))) %>% 
        mutate(fisher.t.ext = NA) %>%  mutate(fisher.t.int = NA) 

    for (i in 1:nrow(df_fisher_species)){
        db_tmp <- df_fisher_species$db[i]
        ctrl <- df_fisher_species$control_type[i]
        tn <- df_fisher_species$taxName[i]
        smpl <- df_fisher_species$sample[i]

        tmp_ext <- c()
        for (ext_c in ctrl_list[[ctrl]]){
            if(ext_c == c(df_fisher_species$sample[i])){next}
            ext_ctrl_species_count <- Asp_species_count %>% 
                filter(sample == ext_c) %>% 
                filter(db == db_tmp) %>% 
                filter(taxName == tn) %>% 
                filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                pull("count")

            ft_ext <- as.numeric(fisher.test(
                            rbind(c(df_fisher_species$count[i],
                                    (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                  c(ext_ctrl_species_count, 
                                    (total_read_count[db_tmp, ext_c])-ext_ctrl_species_count)), 
                            alternative="greater")$p.value)
            tmp_ext <- c(tmp_ext, ft_ext)
            rm(ft_ext)
        }
        df_fisher_species$fisher.t.ext[i] <- mean(tmp_ext, na.rm = TRUE)

        if(ctrl == "Pctrls" & df_fisher_species$sample[i] %nin% c('A05Pctrl2','A05Pctrl3')){
            int_c <- gsub(df_fisher_species$sample[i], pattern = "asp", replacement = "ctrl")
            if(int_c %in% Asp_species_count$sample & int_c %in% colnames(total_read_count)){
                int_ctrl_species_count <- Asp_species_count %>% 
                    filter(sample == ext_c) %>% 
                    filter(db == db_tmp) %>% 
                    filter(taxName == tn) %>% 
                    filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                    pull("count")

                df_fisher_species$fisher.t.int[i] <- as.numeric(fisher.test(
                                rbind(c(df_fisher_species$count[i],
                                        (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                      c(int_ctrl_species_count, 
                                        (total_read_count[db_tmp, int_c])-int_ctrl_species_count)), 
                                alternative="greater")$p.value)
            }
        }
    }
    lst_fisher_species[[fav_db]] <- df_fisher_species
}
saveRDS(lst_fisher_species, paste0(INT_SPL, "lst_fisher_species.Rdata"))

[1] "EPRSc2"
[1] "EPRSFv64MCAspDM"
